<a href="https://colab.research.google.com/github/tomonari-masada/course2025-sml/blob/main/12_dimensionality_reduction_(advanced).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dimensionality reduction （発展編）

* GPUが使えるようにランタイムを設定しておく。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

np.random.seed(0)
torch.manual_seed(0)

%config InlineBackend.figure_format = 'retina'

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/data/sign_mnist_train.csv')
test = pd.read_csv('/content/drive/MyDrive/data/sign_mnist_test.csv')

In [ ]:
# Setting the label and the feature columns
y_train = train.loc[:,'label'].values
X_train = train.loc[:,'pixel1':].values
y_test = test.loc[:,'label'].values
X_test = test.loc[:,'pixel1':].values

## データの前処理
* scikit-learnのドキュメンテーションにあるやり方に倣った。
  * まずtraining set全体での各ピクセルの平均値を引き算し・・・
  * 次に各画像内でのローカルな平均値を引き算する。

In [ ]:
# global centering
train_mean = X_train.mean(axis=0)
X_train_centered = X_train - train_mean
X_test_centered = X_test - train_mean

In [ ]:
# local centering
X_train_centered = X_train_centered - X_train_centered.mean(axis=1).reshape(-1, 1)
X_test_centered = X_test_centered - X_test_centered.mean(axis=1).reshape(-1, 1)

In [ ]:
plt.imshow(X_train_centered[0,:].reshape(28,28), cmap=plt.cm.gray);

## 次元圧縮手法によるデータの再構成(reconstruction)

* 画像を複数描画する関数を定義しておく。

In [ ]:
def plot_gallery(title, images, n_col=3, n_row=3, cmap=plt.cm.gray):
  plt.figure(figsize=(2. * n_col, 2.26 * n_row))
  plt.suptitle(title, size=16)
  for i, comp in enumerate(images):
    plt.subplot(n_row, n_col, i + 1)
    vmax = max(comp.max(), -comp.min())
    plt.imshow(
        comp.reshape(28, 28),
        cmap=cmap,
        interpolation="nearest",
        vmin=-vmax,
        vmax=vmax,
    )
    plt.xticks([])
    plt.yticks([])
  plt.subplots_adjust(0.01, 0.05, 0.99, 0.93, 0.04, 0.);

In [ ]:
plot_gallery("sign mnist", X_train_centered[:9], n_row=3, n_col=3)

In [ ]:
n_components = 30
n_instances, instance_dim = X_train_centered.shape
n_test_instances = X_test_centered.shape[0]

In [ ]:
X_train_torch = torch.tensor(X_train_centered, dtype=torch.float, device="cuda")

bases = torch.randn((n_components, instance_dim), requires_grad=True, device="cuda")
coefficients = torch.randn((n_instances, n_components), requires_grad=True, device="cuda")

optimizer = torch.optim.Adam([bases, coefficients], lr=0.01)
criterion = torch.nn.MSELoss()

batch_size = 1000
n_steps = 0
for epoch in range(1000):
  random_indices = torch.randperm(n_instances)
  for i in range(0, n_instances, batch_size):
    indices = random_indices[i:i+batch_size]
    optimizer.zero_grad()
    reconstruction = torch.softmax(coefficients[indices], dim=1) @ bases
    loss = criterion(reconstruction, X_train_torch[indices])
    loss.backward()
    optimizer.step()
    n_steps += 1
    if n_steps % 1000 == 0:
      print(f"step {n_steps} | loss {loss.item():.4f}")

In [ ]:
components = bases.cpu().detach().numpy()

In [ ]:
n_col = 6
n_row = n_components // n_col + (n_components % n_col != 0)
plot_gallery(
    f"{n_components} components",
    components[:n_components],
    n_row=n_row,
    n_col=n_col,
)

### テストセットに含まれる画像の再構成

In [ ]:
X_test_torch = torch.tensor(X_test_centered, dtype=torch.float, device="cuda")

bases.requires_grad = False # basesは固定する。
coefficients = torch.randn((n_test_instances, n_components), requires_grad=True, device="cuda")

optimizer = torch.optim.Adam([coefficients], lr=0.01)
criterion = torch.nn.MSELoss()

batch_size = 1000
n_steps = 0
for epoch in range(1000):
  random_indices = torch.randperm(n_test_instances)
  for i in range(0, n_test_instances, batch_size):
    indices = random_indices[i:i+batch_size]
    optimizer.zero_grad()
    reconstruction = torch.softmax(coefficients[indices], dim=1) @ bases
    loss = criterion(reconstruction, X_test_torch[indices])
    loss.backward()
    optimizer.step()
    n_steps += 1
    if n_steps % 1000 == 0:
      print(f"step {n_steps} | test reconstruction loss {loss.item():.4f}")

In [ ]:
X_test_recon = (torch.softmax(coefficients, dim=1) @ bases).cpu().detach().numpy()

In [ ]:
n_recon_images = 8
indices = np.random.randint(X_test.shape[0], size=n_recon_images)
plot_gallery("original test data", X_test_centered[indices], n_row=1, n_col=n_recon_images)
plot_gallery("reconstructed test data", X_test_recon[indices], n_row=1, n_col=n_recon_images)